# Remeber to update dictionary 

In [1]:
import httplib2
import json
import sys
import codecs
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import numpy as np

In [2]:
ME_df = pd.read_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\F and E Series update 12020219.xlsx', sheet_name = 'MasterEligible', dtype=str)

In [ ]:
ME_df

In [ ]:
dictionary = {460:'ford',452:'bmw',467 :'chevrolet', 448:'toyota',477:'chrysler',476:'dodge',474:'honda', 480:'infiniti', 478: 'nissan'}

In [22]:
def getMake():
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetAllMakes?format=json')
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getMakeID(Make):
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetAllMakes?format=json')
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    MK_df = pd.DataFrame(result['Results'])
    result_df = MK_df[MK_df['Make_Name'].str.contains(Make)]
    return result_df

def getModel(Make):
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetModelsForMake/%s?format=json' %Make)
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getModel_Year(Make,Year):
    url = ('https://vpic.nhtsa.dot.gov/api/vehicles/GetModelsForMakeYear/make/%s/modelyear/%s?format=json' %(Make,Year))
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getModel_Year_MakeID(MakeID,Year):
    url = ('https://vpic.nhtsa.dot.gov/api//vehicles/GetModelsForMakeIdYear/makeId/%s/modelyear/%s?format=json' %(MakeID,Year))
    h = httplib2.Http()
    response, content = h.request(url, 'GET')
    result = json.loads(content)
    return pd.DataFrame(result['Results'])

def getMatch(Make_id,Year):
    Rlist = []
    try:
        Valid_df = getModel_Year_MakeID(Make_id,Year)

    except:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        Rlist=['No Such year in the Database'] *len(QC_Make_year)
    else:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        for i in range(len(QC_Make_year.Model)):
            str2Match = str(QC_Make_year.Model.values[i])
            strOptions = [z for z in Valid_df.Model_Name]
            Ratios = process.extract(str2Match,strOptions)
            Rlist.append([a + ': ' +str(dict(Ratios)[a]) for a in dict(Ratios)])
    return Rlist
# With score
# def getExactMatch(Make_id,Year):
#     Rlist = []
#     try:
#         Valid_df = getModel_Year_MakeID(Make_id,Year)
        
#     except:
#         QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
#         Rlist=['No Such year in the Database'] *len(QC_Make_year)
#     else:
#         QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
#         for i in range(len(QC_Make_year.Model)):
#             str2Match = str(QC_Make_year.Model.values[i])
#             strOptions = [z for z in Valid_df.Model_Name]
#             Ratios= process.extractOne(str2Match,strOptions, scorer=fuzz.ratio)
#             Rlist.append((':'.join([ str(a) for a in list(Ratios)])))
#     return Rlist

def getTopMatch(Make_id,Year):
    Rlist = []
    try:
        Valid_df = getModel_Year_MakeID(Make_id,Year)
        
    except:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        Rlist=['No Such year in the Database'] *len(QC_Make_year)
    else:
        QC_Make_year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        for i in range(len(QC_Make_year.Model)):
            str2Match = str(QC_Make_year.Model.values[i])
            strOptions = [z for z in Valid_df.Model_Name]
            Ratios= process.extractOne(str2Match,strOptions)
            Rlist.append(( list(Ratios)[0]))
    return Rlist

def getQCtable_OneYear(Make_id,Year):    
    QC_Make_Year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
    QC_Make_Year['TopMatch'] = getTopMatch(Make_id,Year)
    QC_Make_Year['MatchOptions'] = getMatch(Make_id,Year)        
    QC_Make_Year['Match'] = np.where(QC_Make_Year.Model.str.lower() == QC_Make_Year.TopMatch.str.lower(), 'Yes', 'No')
    QC_Make_Year['HighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in QC_Make_Year['MatchOptions']]
    QC_Make_Year['TopMaTchExsting'] = [np.where(QC_Make_Year['TopMatch'].values[i].lower() in QC_Make_Year.Model.str.lower().unique(),'Yes','No') for i in range(len(QC_Make_Year['TopMatch']))]
    QC_Make = pd.concat([QC_Make,QC_Make_Year])
    return QC_Make_Year

def getQCtable_Allyear(Make_id):    
    QC_Make = pd.DataFrame()
    Years = list(ME_df[ME_df.Make.str.lower() == dictionary[Make_id]].Year.unique())
    for Year in Years:
        QC_Make_Year = ME_df[(ME_df.Make.str.lower() == dictionary[Make_id]) & (ME_df.Year == Year)].sort_values(by=['Make','Year','Model'])
        QC_Make_Year['TopMatch'] = getTopMatch(Make_id,Year)
        QC_Make_Year['MatchOptions'] = getMatch(Make_id,Year)        
        QC_Make_Year['Match'] = np.where(QC_Make_Year.Model.str.lower() == QC_Make_Year.TopMatch.str.lower(), 'Yes', 'No')
        QC_Make_Year['HighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in QC_Make_Year['MatchOptions']]
        QC_Make_Year['TopMaTchExsting'] = [np.where(QC_Make_Year['TopMatch'].values[i].lower() in QC_Make_Year.Model.str.lower().unique(),'Yes','No') if QC_Make_Year['HighestScore'].values[i] >= 80 else 'No' for i in range(len(QC_Make_Year['TopMatch']))]
        QC_Make = pd.concat([QC_Make,QC_Make_Year])
    return QC_Make

# def getHightestScores(Make_id,Year):
#     Scores = []
#     if type(getMatch(Make_id,Year)[0]) !=str:
#         for rec in getMatch(Make_id,Year):
#             Scores.append(int(rec[0].split(': ')[1]))
#     else:
#         Scores='N/A'
#     return Scores




## Honda & Acura

In [23]:
getMakeID('Honda')

,Make_ID,Make_Name
3804,474,Honda
7596,7351,Sundiro Honda Motorcycle Co. Ltd


In [ ]:
getMakeID('Acura')

In [38]:
# get relevent data from MasterEligible tabel
target = ME_df[(ME_df.Make.str.lower() == 'honda') | (ME_df.Make.str.lower() == 'acura')].sort_values(by=['Year','Make','Model'])
target

,Make,Model,Year,RyanAddRemoveDate,RyanEligibleFlag,RyanSource,RyanDescription
74,Acura,NSX,1996,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
118,Acura,RL,1996,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
164,Acura,TL,1996,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
4261,Honda,Civic,1996,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
4589,Honda,Odyssey,1996,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
0,Acura,CL,1997,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
18,Acura,EL,1997,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
75,Acura,NSX,1997,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
119,Acura,RL,1997,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
165,Acura,TL,1997,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3


In [39]:
def getMatch(Make_id,Year):
    Rlist = []
    try:
        Valid_df = getModel_Year_MakeID(Make_id,Year)

    except:
        QC_Make_year = target[(target.Year == Year) & (target.Make.str.lower() == 'honda')].sort_values(by=['Year','Make','Model'])
        Rlist=['No Such year in the Database'] *len(QC_Make_year)
    else:
        QC_Make_year = target[(target.Year == Year) & (target.Make.str.lower() == 'honda')].sort_values(by=['Year','Make','Model'])
        for i in range(len(QC_Make_year.Model)):
            str2Match = str(QC_Make_year.Model.values[i])
            strOptions = [z for z in Valid_df.Model_Name]
            Ratios = process.extract(str2Match,strOptions)
            Rlist.append([a + ': ' +str(dict(Ratios)[a]) for a in dict(Ratios)])
    return Rlist


def getHondaQCtable_Allyear():    
    QC_Make = pd.DataFrame()
    Years = list(target[target.Make.str.lower() == 'honda'].Year.unique())
    for Year in Years:
        temp1 = getMatch(474,Year)
        temp2 = getMatch(475,Year)
        QC_Make_Year = target[(target.Year == Year) & (target.Make.str.lower() == 'honda')].sort_values(by=['Year','Make','Model'])
        QC_Make_Year['TopMatch'] = [i[0].split(':')[0] for i in temp1]
        QC_Make_Year['MatchOptions'] = temp1        
        QC_Make_Year['Match'] = np.where(QC_Make_Year.Model.str.lower() == QC_Make_Year.TopMatch.str.lower(), 'Yes', 'No')
        QC_Make_Year['HighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in QC_Make_Year['MatchOptions']]
        QC_Make_Year['TopMaTchExsting'] = [np.where(QC_Make_Year['TopMatch'].values[i].lower() in QC_Make_Year.Model.str.lower().unique(),'Yes','No') if QC_Make_Year['HighestScore'].values[i] != 'N/A' and QC_Make_Year['HighestScore'].values[i] >=80  else 'No' for i in range(len(QC_Make_Year['TopMatch']))]
        QC_Make_Year['AlternativeTopMatch'] = [i[0].split(':')[0] for i in temp2]
        QC_Make_Year['AlternativeHighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in temp2]
        QC_Make_Year['AlternativeExsiting'] =[np.where(QC_Make_Year['AlternativeTopMatch'].values[i].lower() in target[(target.Make.str.lower() == 'acura') & (target.Year == Year)].Model.str.lower().unique(),'Yes','No') if QC_Make_Year['AlternativeHighestScore'].values[i] != 'N/A' and QC_Make_Year['AlternativeHighestScore'].values[i] >= 80  else 'No' for i in range(len(QC_Make_Year['TopMatch']))]
        QC_Make = pd.concat([QC_Make,QC_Make_Year])
    return QC_Make

cross_Honda = getHondaQCtable_Allyear()


cross_sorted = cross_Honda.sort_values(by=['HighestScore','AlternativeHighestScore','Make','Model','Year'],ascending = [False,False,True,True,True])
cross_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\test\\cross_Honda.xlsx',index = False)

In [40]:
def getMatch(Make_id,Year):
    Rlist = []
    try:
        Valid_df = getModel_Year_MakeID(Make_id,Year)

    except:
        QC_Make_year = target[(target.Year == Year) & (target.Make.str.lower() == 'acura')].sort_values(by=['Year','Make','Model'])
        Rlist=['No Such year in the Database'] *len(QC_Make_year)
    else:
        QC_Make_year = target[(target.Year == Year) & (target.Make.str.lower() == 'acura')].sort_values(by=['Year','Make','Model'])
        for i in range(len(QC_Make_year.Model)):
            str2Match = str(QC_Make_year.Model.values[i])
            strOptions = [z for z in Valid_df.Model_Name]
            Ratios = process.extract(str2Match,strOptions)
            Rlist.append([a + ': ' +str(dict(Ratios)[a]) for a in dict(Ratios)])
    return Rlist


def getAcuraQCtable_Allyear():    
    QC_Make = pd.DataFrame()
    Years = list(target[(target.Make.str.lower() == 'acura')].Year.unique())
    for Year in Years:
        temp1 = getMatch(475,Year)
        temp2 = getMatch(474,Year)        
        QC_Make_Year = target[(target.Year == Year) & (target.Make.str.lower() == 'acura')].sort_values(by=['Year','Make','Model'])
        QC_Make_Year['TopMatch'] = [i[0].split(':')[0] for i in temp1]
        QC_Make_Year['MatchOptions'] = temp1        
        QC_Make_Year['Match'] = np.where(QC_Make_Year.Model.str.lower() == QC_Make_Year.TopMatch.str.lower(), 'Yes', 'No')
        QC_Make_Year['HighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in QC_Make_Year['MatchOptions']]
        QC_Make_Year['TopMaTchExsting'] = [np.where(QC_Make_Year['TopMatch'].values[i].lower() in QC_Make_Year.Model.str.lower().unique(),'Yes','No') if QC_Make_Year['HighestScore'].values[i] != 'N/A' and  QC_Make_Year['HighestScore'].values[i] >= 80 else 'No' for i in range(len(QC_Make_Year['TopMatch']))]
        QC_Make_Year['AlternativeTopMatch'] = [i[0].split(':')[0] for i in temp2]
        QC_Make_Year['AlternativeHighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in temp2]
        QC_Make_Year['AlternativeExsiting'] =[np.where(QC_Make_Year['AlternativeTopMatch'].values[i].lower() in target[(target.Make.str.lower() == 'honda') & (target.Year == Year)].Model.str.lower().unique(),'Yes','No') if QC_Make_Year['AlternativeHighestScore'].values[i] != 'N/A' and QC_Make_Year['AlternativeHighestScore'].values[i] >= 80 else 'No' for i in range(len(QC_Make_Year['TopMatch']))]
        QC_Make = pd.concat([QC_Make,QC_Make_Year])
    return QC_Make

cross_Acura = getAcuraQCtable_Allyear()

cross_sorted = cross_Acura.sort_values(by=['HighestScore','AlternativeHighestScore','Make','Model','Year'],ascending = [False,False,True,True,True])
cross_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\test\\cross_Acura.xlsx',index = False)




## Toyota & Lexus

In [30]:
getMakeID('Toyota')

,Make_ID,Make_Name
8120,448,Toyota


In [31]:
getMakeID('Lexus')

,Make_ID,Make_Name
4762,515,Lexus


In [32]:
# get relevent data from MasterEligible tabel
target = ME_df[(ME_df.Make.str.lower() == 'toyota') | (ME_df.Make.str.lower() == 'lexus')].sort_values(by=['Year','Make','Model'])
target

,Make,Model,Year,RyanAddRemoveDate,RyanEligibleFlag,RyanSource,RyanDescription
10187,Toyota,Previa,1990,2019-09-25 16:03:15.593000,Y,Round 4 Eligible Vehicle List 09-18-2019.XLSX ...,Round 4
10188,Toyota,Previa,1991,2019-09-25 16:03:15.593000,Y,Round 4 Eligible Vehicle List 09-18-2019.XLSX ...,Round 4
6604,Lexus,SC,1992,2019-09-25 16:03:15.593000,Y,Round 4 Eligible Vehicle List 09-18-2019.XLSX ...,Round 4
10189,Toyota,Previa,1992,2019-09-25 16:03:15.593000,Y,Round 4 Eligible Vehicle List 09-18-2019.XLSX ...,Round 4
6605,Lexus,SC,1993,2019-09-25 16:03:15.593000,Y,Round 4 Eligible Vehicle List 09-18-2019.XLSX ...,Round 4
10190,Toyota,Previa,1993,2019-09-25 16:03:15.593000,Y,Round 4 Eligible Vehicle List 09-18-2019.XLSX ...,Round 4
6606,Lexus,SC,1994,2019-09-25 16:03:15.593000,Y,Round 4 Eligible Vehicle List 09-18-2019.XLSX ...,Round 4
10191,Toyota,Previa,1994,2019-09-25 16:03:15.593000,Y,Round 4 Eligible Vehicle List 09-18-2019.XLSX ...,Round 4
6362,Lexus,LS,1995,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
6607,Lexus,SC,1995,2019-09-25 16:03:15.593000,Y,Round 4 Eligible Vehicle List 09-18-2019.XLSX ...,Round 4


In [36]:
def getMatch(Make_id,Year):
    Rlist = []
    try:
        Valid_df = getModel_Year_MakeID(Make_id,Year)

    except:
        QC_Make_year = target[(target.Year == Year) & (target.Make.str.lower() == 'toyota')].sort_values(by=['Year','Make','Model'])
        Rlist=['No Such year in the Database'] *len(QC_Make_year)
    else:
        QC_Make_year = target[(target.Year == Year) & (target.Make.str.lower() == 'toyota')].sort_values(by=['Year','Make','Model'])
        for i in range(len(QC_Make_year.Model)):
            str2Match = str(QC_Make_year.Model.values[i])
            strOptions = [z for z in Valid_df.Model_Name]
            Ratios = process.extract(str2Match,strOptions)
            Rlist.append([a + ': ' +str(dict(Ratios)[a]) for a in dict(Ratios)])
    return Rlist


def getToyotaQCtable_Allyear():    
    QC_Make = pd.DataFrame()
    Years = list(target[target.Make.str.lower() == 'toyota'].Year.unique())
    for Year in Years:
        temp1 = getMatch(448,Year)
        temp2 = getMatch(515,Year)
        QC_Make_Year = target[(target.Year == Year) & (target.Make.str.lower() == 'toyota')].sort_values(by=['Year','Make','Model'])
        QC_Make_Year['TopMatch'] = [i[0].split(':')[0] for i in temp1]
        QC_Make_Year['MatchOptions'] = temp1        
        QC_Make_Year['Match'] = np.where(QC_Make_Year.Model.str.lower() == QC_Make_Year.TopMatch.str.lower(), 'Yes', 'No')
        QC_Make_Year['HighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in QC_Make_Year['MatchOptions']]
        QC_Make_Year['TopMaTchExsting'] = [np.where(QC_Make_Year['TopMatch'].values[i].lower() in QC_Make_Year.Model.str.lower().unique(),'Yes','No') if QC_Make_Year['HighestScore'].values[i] != 'N/A' and QC_Make_Year['HighestScore'].values[i] >=80  else 'No' for i in range(len(QC_Make_Year['TopMatch']))]
        QC_Make_Year['AlternativeTopMatch'] = [i[0].split(':')[0] for i in temp2]
        QC_Make_Year['AlternativeHighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in temp2]
        QC_Make_Year['AlternativeExsiting'] =[np.where(QC_Make_Year['AlternativeTopMatch'].values[i].lower() in target[(target.Make.str.lower() == 'lexus') & (target.Year == Year)].Model.str.lower().unique(),'Yes','No') if QC_Make_Year['AlternativeHighestScore'].values[i] != 'N/A' and QC_Make_Year['AlternativeHighestScore'].values[i] >= 80  else 'No' for i in range(len(QC_Make_Year['TopMatch']))]
        QC_Make = pd.concat([QC_Make,QC_Make_Year])
    return QC_Make

cross_Toyota = getToyotaQCtable_Allyear()


cross_sorted = cross_Toyota.sort_values(by=['HighestScore','AlternativeHighestScore','Make','Model','Year'],ascending = [False,False,True,True,True])
cross_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\test\\cross_Toyota.xlsx',index = False)

In [37]:
def getMatch(Make_id,Year):
    Rlist = []
    try:
        Valid_df = getModel_Year_MakeID(Make_id,Year)

    except:
        QC_Make_year = target[(target.Year == Year) & (target.Make.str.lower() == 'lexus')].sort_values(by=['Year','Make','Model'])
        Rlist=['No Such year in the Database'] *len(QC_Make_year)
    else:
        QC_Make_year = target[(target.Year == Year) & (target.Make.str.lower() == 'lexus')].sort_values(by=['Year','Make','Model'])
        for i in range(len(QC_Make_year.Model)):
            str2Match = str(QC_Make_year.Model.values[i])
            strOptions = [z for z in Valid_df.Model_Name]
            Ratios = process.extract(str2Match,strOptions)
            Rlist.append([a + ': ' +str(dict(Ratios)[a]) for a in dict(Ratios)])
    return Rlist


def getLexusQCtable_Allyear():    
    QC_Make = pd.DataFrame()
    Years = list(target[(target.Make.str.lower() == 'lexus')].Year.unique())
    for Year in Years:
        temp1 = getMatch(515,Year)
        temp2 = getMatch(448,Year)        
        QC_Make_Year = target[(target.Year == Year) & (target.Make.str.lower() == 'lexus')].sort_values(by=['Year','Make','Model'])
        QC_Make_Year['TopMatch'] = [i[0].split(':')[0] for i in temp1]
        QC_Make_Year['MatchOptions'] = temp1        
        QC_Make_Year['Match'] = np.where(QC_Make_Year.Model.str.lower() == QC_Make_Year.TopMatch.str.lower(), 'Yes', 'No')
        QC_Make_Year['HighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in QC_Make_Year['MatchOptions']]
        QC_Make_Year['TopMaTchExsting'] = [np.where(QC_Make_Year['TopMatch'].values[i].lower() in QC_Make_Year.Model.str.lower().unique(),'Yes','No') if QC_Make_Year['HighestScore'].values[i] != 'N/A' and  QC_Make_Year['HighestScore'].values[i] >= 80 else 'No' for i in range(len(QC_Make_Year['TopMatch']))]
        QC_Make_Year['AlternativeTopMatch'] = [i[0].split(':')[0] for i in temp2]
        QC_Make_Year['AlternativeHighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in temp2]
        QC_Make_Year['AlternativeExsiting'] =[np.where(QC_Make_Year['AlternativeTopMatch'].values[i].lower() in target[(target.Make.str.lower() == 'toyota') & (target.Year == Year)].Model.str.lower().unique(),'Yes','No') if QC_Make_Year['AlternativeHighestScore'].values[i] != 'N/A' and QC_Make_Year['AlternativeHighestScore'].values[i] >= 80 else 'No' for i in range(len(QC_Make_Year['TopMatch']))]
        QC_Make = pd.concat([QC_Make,QC_Make_Year])
    return QC_Make

cross_Lexus = getLexusQCtable_Allyear()

cross_sorted = cross_Lexus.sort_values(by=['HighestScore','AlternativeHighestScore','Make','Model','Year'],ascending = [False,False,True,True,True])
cross_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\test\\cross_Lexus.xlsx',index = False)




## Nissan & Infiniti

In [ ]:
getMakeID('Nissan')

In [ ]:
getMakeID('Infiniti')

In [7]:
# get relevent data from MasterEligible tabel
target = ME_df[(ME_df.Make.str.lower() == 'nissan') | (ME_df.Make.str.lower() == 'infiniti')| (ME_df.Make.str.lower() == 'infinity')].sort_values(by=['Year','Make','Model'])
target

,Make,Model,Year,RyanAddRemoveDate,RyanEligibleFlag,RyanSource,RyanDescription
8206,Nissan,300ZX,1995,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
8207,Nissan,300ZX,1996,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
5142,Infiniti,Q45,1997,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
8352,Nissan,Frontier,1997,2018-05-17 16:53:40.733000,Y,round 3 list per Denise,Round 3
8262,Nissan,Altima,1998,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
8353,Nissan,Frontier,1998,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
8690,Nissan,Sentra,1998,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
8263,Nissan,Altima,1999,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
8354,Nissan,Frontier,1999,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2
8691,Nissan,Sentra,1999,2017-11-30 00:00:00,Y,autopartsclass.com List,Round 1 & 2


In [8]:
def getMatch(Make_id,Year):
    Rlist = []
    try:
        Valid_df = getModel_Year_MakeID(Make_id,Year)

    except:
        QC_Make_year = target[(target.Year == Year) & (target.Make.str.lower() == 'nissan')].sort_values(by=['Year','Make','Model'])
        Rlist=['No Such year in the Database'] *len(QC_Make_year)
    else:
        QC_Make_year = target[(target.Year == Year) & (target.Make.str.lower() == 'nissan')].sort_values(by=['Year','Make','Model'])
        for i in range(len(QC_Make_year.Model)):
            str2Match = str(QC_Make_year.Model.values[i])
            strOptions = [z for z in Valid_df.Model_Name]
            Ratios = process.extract(str2Match,strOptions)
            Rlist.append([a + ': ' +str(dict(Ratios)[a]) for a in dict(Ratios)])
    return Rlist


def getNissanQCtable_Allyear():    
    QC_Make = pd.DataFrame()
    Years = list(target[target.Make.str.lower() == 'nissan'].Year.unique())
    for Year in Years:
        temp1 = getMatch(478,Year)
        temp2 = getMatch(480,Year)
        QC_Make_Year = target[(target.Year == Year) & (target.Make.str.lower() == 'nissan')].sort_values(by=['Year','Make','Model'])
        QC_Make_Year['TopMatch'] = [i[0].split(':')[0] for i in temp1]
        QC_Make_Year['MatchOptions'] = temp1        
        QC_Make_Year['Match'] = np.where(QC_Make_Year.Model.str.lower() == QC_Make_Year.TopMatch.str.lower(), 'Yes', 'No')
        QC_Make_Year['HighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in QC_Make_Year['MatchOptions']]
        QC_Make_Year['TopMaTchExsting'] = [np.where(QC_Make_Year['TopMatch'].values[i].lower() in QC_Make_Year.Model.str.lower().unique(),'Yes','No') if QC_Make_Year['HighestScore'].values[i] >= 80 else 'No' for i in range(len(QC_Make_Year['TopMatch']))]
        QC_Make_Year['AlternativeTopMatch'] = [i[0].split(':')[0] for i in temp2]
        QC_Make_Year['AlternativeHighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in temp2]
        QC_Make_Year['AlternativeExsiting'] =[np.where(QC_Make_Year['AlternativeTopMatch'].values[i].lower() in target[((target.Make.str.lower() == 'infiniti')|(target.Make.str.lower() == 'infinity')) & (target.Year == Year)].Model.str.lower().unique(),'Yes','No') if QC_Make_Year['AlternativeHighestScore'].values[i] >= 80 else 'No' for i in range(len(QC_Make_Year['TopMatch']))]
        QC_Make = pd.concat([QC_Make,QC_Make_Year])
    return QC_Make

cross_Nissan = getNissanQCtable_Allyear()
cross_Nissan

cross_sorted = cross_Nissan.sort_values(by=['HighestScore','AlternativeHighestScore','Make','Model','Year'],ascending = [False,False,True,True,True])
cross_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\test\\cross_Nissan.xlsx',index = False)

TypeError: '>=' not supported between instances of 'str' and 'int'

In [ ]:
def getMatch(Make_id,Year):
    Rlist = []
    try:
        Valid_df = getModel_Year_MakeID(Make_id,Year)

    except:
        QC_Make_year = target[(target.Year == Year) & ((target.Make.str.lower() == 'infiniti')|(target.Make.str.lower() == 'infinity'))].sort_values(by=['Year','Make','Model'])
        Rlist=['No Such year in the Database'] *len(QC_Make_year)
    else:
        QC_Make_year = target[(target.Year == Year) & ((target.Make.str.lower() == 'infiniti')|(target.Make.str.lower() == 'infinity'))].sort_values(by=['Year','Make','Model'])
        for i in range(len(QC_Make_year.Model)):
            str2Match = str(QC_Make_year.Model.values[i])
            strOptions = [z for z in Valid_df.Model_Name]
            Ratios = process.extract(str2Match,strOptions)
            Rlist.append([a + ': ' +str(dict(Ratios)[a]) for a in dict(Ratios)])
    return Rlist


def getInfinitiQCtable_Allyear():    
    QC_Make = pd.DataFrame()
    Years = list(target[(target.Make.str.lower() == 'infiniti')|(target.Make.str.lower() == 'infinity')].Year.unique())
    for Year in Years:
        temp1 = getMatch(480,Year)
        temp2 = getMatch(478,Year)        
        QC_Make_Year = target[(target.Year == Year) & ((target.Make.str.lower() == 'infiniti')|(target.Make.str.lower() == 'infinity'))].sort_values(by=['Year','Make','Model'])
        QC_Make_Year['TopMatch'] = [i[0].split(':')[0] for i in temp1]
        QC_Make_Year['MatchOptions'] = temp1        
        QC_Make_Year['Match'] = np.where(QC_Make_Year.Model.str.lower() == QC_Make_Year.TopMatch.str.lower(), 'Yes', 'No')
        QC_Make_Year['HighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in QC_Make_Year['MatchOptions']]
        QC_Make_Year['TopMaTchExsting'] = [np.where(QC_Make_Year['TopMatch'].values[i].lower() in QC_Make_Year.Model.str.lower().unique(),'Yes','No') if QC_Make_Year['HighestScore'].values[i] >= 80 else 'No' for i in range(len(QC_Make_Year['TopMatch']))]
        QC_Make_Year['AlternativeTopMatch'] = [i[0].split(':')[0] for i in temp2]
        QC_Make_Year['AlternativeHighestScore'] = [int(rec[0].split(': ')[1]) if type(rec) != str else 'N/A' for rec in temp2]
        QC_Make_Year['AlternativeExsiting'] =[np.where(QC_Make_Year['AlternativeTopMatch'].values[i].lower() in target[(target.Make.str.lower() == 'nissan') & (target.Year == Year)].Model.str.lower().unique(),'Yes','No') if QC_Make_Year['AlternativeHighestScore'].values[i] >= 80 else 'No' for i in range(len(QC_Make_Year['TopMatch']))]
        QC_Make = pd.concat([QC_Make,QC_Make_Year])
    return QC_Make

cross_Infinit = getInfinitiQCtable_Allyear()
cross_Infinit

cross_sorted = cross_Infinit.sort_values(by=['HighestScore','AlternativeHighestScore','Make','Model','Year'],ascending = [False,False,True,True,True])
cross_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\test\\cross_Infinit.xlsx',index = False)


